### Libraries

In [13]:
import math
import numpy as np

# import pandas 
# import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

### Constants

In [14]:
# @title Gauss-Laguerre constants
# Notation: (weight, abscissa)

GAUSS_LAGUERRE = [
    (0.458964, 0.222847),
    (0.417, 1.188932),
    (0.113373, 2.992736),
    (0.0103992, 5.775144),
    (0.000261017, 9.837467),
    (0.000000898548, 15.982874)
]

### PDF, CDF of Weibull distribution

In [15]:
# @title Weibull distribution function
# Uses Gauss-Laguerre
def f_Weibull (t, k, mu):
    denomArea = 0
    
    for node in GAUSS_LAGUERRE:
        denomArea += node[0] * (node[1]**(1/k))

    lbda = mu / denomArea

    return (k/lbda) * ((t/lbda)**(k-1)) * (math.e ** (-(t/lbda)**k))

In [16]:
# @title Integral of Weibull function
# using Gauss-Legendre; a=0, b=1
def cdf (t, k, mu):
    area = 0
    
    for node in GAUSS_LAGUERRE:
        area += node[0] * f_Weibull(t + node[1], k, mu) * (math.e**node[1])

    return 1 - area

In [17]:
# @title CDF values
t_values = []
y_CDF_05 = []
y_CDF_1 = []
y_CDF_2 = []

for t in range(0, 251, 1):
    t /= 100
    t_values.append(t)
    y_CDF_05.append(cdf(t, 0.5, 2))
    y_CDF_1.append(cdf(t, 1, 1))
    y_CDF_2.append(cdf(t, 2, math.sqrt(math.pi)/2))

In [18]:
# @title CDF plotting
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=t_values, y=y_CDF_05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
    go.Scatter(x=t_values, y=y_CDF_1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
    go.Scatter(x=t_values, y=y_CDF_2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
])
fig.update_layout(
    height=1080*0.5,
    width=1920*0.6,
    xaxis_title="t",
    yaxis_title="alpha"
)
fig.show()


### Root-finding problem

In [19]:
# @title Inverse survivability
# solved as root-finding problem using bisection method
def surv_inv (alpha, a, b, k, mu):
    isDiffSign = lambda x1, x2 : np.sign(x1) != np.sign(x2)

    f = lambda t : cdf(t, k, mu) - alpha

    tol = 1.0e-9

    fa = f(a)
    fb = f(b)

    if fa == 0.0:
        return a
    
    if fb == 0.0:
        return b
    
    if not isDiffSign(fa, fb):
        return None
    
    n = int (math.ceil (math.log(abs(b-a) / tol) / math.log(2.0)))

    # print("---> iterations: ", n)

    for i in range(n):
        # print("-> CDF(" + str(a) + ") - " + str(alpha) + " = " + str(fa))
        # print("-> CDF(" + str(b) + ") - " + str(alpha) + " = " + str(fb))

        c = 0.5 * (a + b)
        fc = f(c)

        if fc == 0.0:
            # print("---> returning c =", c)
            return c

        if isDiffSign(fa, fc):
            b = c
            fb = fc
        
        elif isDiffSign(fb, fc):
            a = c
            fa = fc

    return 0.5 * (a+b)

### Evaluation

In [20]:
# List for values storing
x_values = []
y_t_05 = []
y_t_1 = []
y_t_2 = []

# Average life expectancy
mu = 78

In [21]:
# For alpha in (0,1) with 0.01 step
for alpha in range(1, 101, 1):
    alpha /= 100

    x_values.append(alpha)

    y_t_05.append(surv_inv(alpha, 0, 2.5, 0.5, 2))
    y_t_1.append(surv_inv(alpha, 0, 2.5, 1, 1))
    y_t_2.append(surv_inv(alpha, 0, 2.5, 2, math.sqrt(math.pi)/2))

### Plotting

In [22]:
# @title Inverse survivability plot
fig = go.Figure()
fig.add_traces([
    go.Scatter(x=x_values, y=y_t_05, mode='lines', marker = {'color' : 'blue'}, name="k = 0.5"),
    go.Scatter(x=x_values, y=y_t_1, mode='lines', marker = {'color' : 'red'}, name="k = 1"),
    go.Scatter(x=x_values, y=y_t_2, mode='lines', marker = {'color' : 'magenta'}, name="k = 2")
])
fig.update_layout(
    height=1080*0.5,
    width=1920*0.5,
    xaxis_title="alpha",
    yaxis_title="t"
)
fig.show()

In [23]:
# print(x_values)
# print(y_t_05)
# print(y_t_1)
# print(y_t_2)